In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
def msmt_association(measurements, map_hypothesis, thresh=None):
    """
    Parameters:
    -----------
    measurements: shape: (n_measurements, 2+1)
        From the m-th camera, 
    map_hypothesis: shape: (n_targets, 2+1)
        State hypothesis on cameras at t-1 of all tracked objects, 

    RETURN:
    msmt_m: 
        Associated measurements of m-th camera, order in the same way as the targets. |msmt_m[i] <-> tracks[i]|
    """

    msmt_m = []
    argmaxes = []
    for hi in map_hypothesis:
        w = util.squared_error(measurements, hi,sigma=20)
        argmax_w = np.argmax(w)
        argmaxes.append(argmax_w)
        msmt_m.append(measurements[argmax_w])
        # measurements = np.delete(measurements, argmax_w, axis=0)
        # print(w, argmax_w)
    
    return np.array(msmt_m), argmaxes

def observe_fn(cam_group, particles):
        """
        Parameters:
            @ particles : all particles of a track
        Returns:
            @ hypothesis from x using the camera projection matrix
            This returns list of arrays which corresponds to hypothesis of particles on each camera
            |hypo[i] <-> camera[i]|
        """

        pos = particles[:, 0:3]
        labels = particles[:, -1]
        hypothesis = cam_group.get_group_measurement(pos, labels)

        return hypothesis

def prior_fn(n_particles, spawn_regions=None):
        if spawn_regions is not None:
            n_spawn = spawn_regions.shape[0]
            for region in spawn_regions:
                pos = np.random.normal(region, 2*np.ones(3), (n_particles, 3))
                vel = np.random.normal(np.zeros(3), np.ones(1), (n_particles, 3))
                # label = np.random.randint(0, 2, (n,1))
                particles = np.hstack([pos, vel])
            return particles
        else:
            pos = np.random.uniform(low=[-20, -20, 5], high=[20, 20, 25], size=(n_particles,3))
            vel = np.random.uniform(low=np.zeros(3), high=np.ones(3), size=(n_particles,3))

            # label = np.random.randint(0, 2, (n,1))
            particles = np.hstack([pos, vel])
            return particles

def weight_fn(msmt, hypo, sigma, verbose=False):
        """
        Iterate through each camera, update weights
        @ msmt: [{yj}_0, {yj}_1, ...{yj}_n]    n cameras
        @ hypo: [bearing_0, bearing_1, ....bearing_n]   n_cameras
        """

        n_particles = hypo[0].shape[0]
        weights = np.ones(n_particles)

        for zi, hi in zip(msmt, hypo):
            if zi[0] < 0: # got no measurements so weights are unchanged
                wi = np.ones(n_particles)
            wi = util.squared_error(hi, zi, sigma=sigma)
            weights *= wi
        weights_sum = np.sum(weights)

        print(weights_sum)
        return weights / weights_sum

def resample(n_eff, particles, weights, resample_fn):
    if n_eff < n_eff_thresh:
        indices = resample_fn(weights)
        particles = particles[indices, :]
        weights = np.ones(n_particles) / n_particles
    
    return particles, weights

dt = 1/10
A = np.block([
    [np.eye(3), dt*np.eye(3)],
    [np.zeros((3,3)), np.eye(3)]
])
A = np.block([
    [A, np.zeros((6,1))],
    [np.zeros((1,6)), 1]
])

def dynamics_d(x):
    """
    Discrete dynamics, last state is the target label
    """
    n, d = x.shape
    w = np.zeros((d, n))
    w[3:6] = np.random.normal(0, 0.1, (3,n))
    x_1 = A @ x.T + w

    return x_1.T


In [3]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf

    return start + v_max * t

def target_stationary(start):
    return start

In [11]:
class Track:
    """
    This is a data structure for representing tracked targets
    """
    def __init__(self, n_particles: int, x0=None, label=None):
        self.n_particles = n_particles
        self.mean = None
        self.cov = None
        self.weights = np.ones(n_particles)
        self.particles = None
        self.label = label
        self.trajectory = []
        self.n_eff_thresh = n_eff_thresh
        
        self.cam_group = None
        self.hypothesis = None

        if x0 is not None:
            # x0 = x0 + np.random.normal(np.zeros(3), np.ones(3)) * 0
            x0 = x0
            pos = np.random.normal(x0, [1,1,1], size=(n_particles, 3))
            vel = np.random.normal([0.2,0,0], np.zeros(3), size=(n_particles, 3))
            labels = np.ones((n_particles,1)) * label
            self.particles = np.hstack([pos, vel, labels])
            self.mean_state = x0
        else:
            pos = np.random.uniform([-20,-20,0], [20,20,25], size=(n_particles, 3))
            vel = np.random.normal([0,0,0], [1,1,1], size=(n_particles, 3))
            self.particles = np.hstack([pos, vel])
    
    def update(self, weights, particles, resample_fn):
        """
        Update track after particle filter update
        """
        self.particles = particles
        # normalizing weight
        self.weight_sum = np.sum(weights)
        self.weights = weights / self.weight_sum
        
        # mean and cov
        self.mean_state = np.sum(self.particles.T * self.weights, axis=-1).T
        self.cov_state = np.cov(self.particles, rowvar=False, aweights=self.weights)
        
        self.trajectory.append(self.mean_state[0:3])
                
        indices = resample_fn(weights)
        self.particles = self.particles[indices, :]
        self.weights = np.ones(self.n_particles) / self.n_particles
        
        return particles, weights
    
    def set_cam_group(self, cam_group: camera.CamGroup):
        self.cam_group = cam_group

In [12]:
cam_param = [642.0926, 642.0926, 1000.5, 1000.5,0]
# [x y z roll pitch yaw]
cam_poses = np.array([
    [20, 20, 12, 0, 0, -2.2],
    [20, -20, 12, 0, 0, 2.2],
    [-20, -20, 12, 0, 0, 0.7],
    [-20, 20, 12, 0, 0, -0.7],
])

cam_group = camera.CamGroup(cam_param, cam_poses[0:4,:])

n_targets = 2
tracks = []

t_final = 200

verbose = False
n_particles = 500

x0_1 = np.array([-20, 2, 20])
x0_2 = np.array([20, -2, 20])
x0_3 = np.array([5, 20, 15])

n_eff_thresh = 1
track1 = Track(n_particles, x0=x0_1, label=1)
track2 = Track(n_particles, x0=x0_2, label=1)
track3 = None

tracks = [track1, track2]
tracks_true = np.hstack([x0_1, x0_2])

state_est1 = []
state_est2 = []

def resample(weights_):
    n = len(weights_)
    positions = np.arange(n)

    indices = [np.random.choice(a=n, p=weights_, replace=True) for i in range(n)]
    return indices

association_order = []

for t in range(t_final):
    target_1 = target_traj_straight(t, x0_1, x0_1+[40,0,0], t_final)
    target_2 = target_traj_straight(t, x0_2, x0_2+[-40,0,0], t_final)

    # target_2 = target_traj_circle(t)
    # target_1 = target_stationary(x0_1)
    # target_2 = target_stationary(x0_2)
    targets = np.vstack([target_1, target_2])
    tracks_true = np.vstack([tracks_true, targets.reshape(1,6)])
    map
    # if t > 100:
    #     target_3 = target_traj_straight(t-100, x0_3, x0_3+[0,-30,0], t_final-50)
    #     targets = np.vstack([targets, target_3])
    #     if track3 is None:
    #         track3 = Track(n_particles, target_3, 0)
    #         tracks.append(track3)

    #each camera can give multiple measurements
    z = cam_group.get_group_measurement(targets, labels=[1,1, 0])

    # map_states = [track.map_state[0:3] for track in tracks]
    mean_states = [track.mean_state[0:3] for track in tracks]
    
    # map_hypo = cam_group.get_group_measurement(map_states, labels=[1,1, 0])
    mean_hypo = cam_group.get_group_measurement(mean_states, labels=[1,1, 0])

    z_a = []
    msmt_order = []
    for z_m, mean_hypo_m in zip(z, mean_hypo):
        z_a_m, order = msmt_association(z_m, mean_hypo_m)
        z_a.append(z_a_m)
        msmt_order.append(order)
    association_order.append(msmt_order)

    for i, track in enumerate(tracks):
        msmt = [z_a_m[i] for z_a_m in z_a]
        particles = dynamics_d(track.particles)
        hypo = observe_fn(cam_group, track.particles)
        weights = weight_fn(msmt=msmt, hypo=hypo, sigma=100)
        track.update(weights, particles, resample)


1.056402828244047e-07
7.572507153153029e-08
1.122076662801225e-07
8.220211943276358e-08
1.1561979007549487e-07
8.889942743980825e-08
1.1532863498498801e-07
7.92205589598769e-08
1.1586565174357422e-07
8.991285768253538e-08
1.1671134543417085e-07
9.700018013237518e-08
1.135597347266053e-07
8.334543182902021e-08
1.1296762344214015e-07
9.058429205006772e-08
2.170317154720296e-74
9.011964924181543e-08
1.6986914414797773e-10


/tmp/ipykernel_3064917/924757520.py:42: RuntimeWarning: Degrees of freedom <= 0 for slice
  self.cov_state = np.cov(self.particles, rowvar=False, aweights=self.weights)
/home/zpyang/anaconda3/envs/visnet/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/zpyang/anaconda3/envs/visnet/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


7.978430098183713e-08
1.7908917299846056e-20
8.105256219234843e-08
1.0840706894802366e-20
5.9501213920277577e-08
3.2061627371564643e-21
7.176785468652863e-08
3.1062367939012663e-21
6.499475380472947e-08
1.1413114398259555e-21
5.5081654153693346e-08
2.0182980107055613e-22
5.454132166721651e-08
3.2155886076022767e-22
4.934702856514594e-08
2.193652546832248e-23
4.871239185609251e-08
3.5787212049012205e-10
4.0371204237652554e-08
2.002883389325213e-08
2.989894359412156e-08
3.432422499731757e-08
3.2854773048932944e-08
2.8392705271325647e-08
2.7119829593475675e-08
2.091391930527061e-08
2.6551818708292177e-08
1.8288917210091503e-08
2.9528503135028973e-08
1.9240632095013037e-08
3.6183863973270795e-08
1.9102388592026716e-08
2.6331397966304917e-08
1.8946967511334487e-08
3.2432411359251995e-08
1.7492937119326385e-08
2.5787790193060135e-08
2.8510863178552318e-08
2.864993188389992e-08
1.747864656613019e-08
2.6326448990282502e-08
1.5519234816858196e-08
2.2465945405096945e-08
1.7926537788809425e-08
1.

In [13]:
association_order = np.array(association_order)
print(association_order.shape)
association_count = np.sum(association_order, axis=1)
track1_rate = (4 - association_count[:, 0]) / 4.0 
track2_rate = association_count[:, 1]  / 4.0

traj_1 = np.array(track1.trajectory)
traj_2 = np.array(track2.trajectory)

fig2 = plt.figure("Association accuracy and Track Error", figsize=(20,10))
ax1 = fig2.add_subplot(2,2,1)
ax1.plot(track1_rate)
ax1.set_ylim(0, 1.1)
ax1.set_ylabel("Association Accuracy")
ax1.set_title("Track 1")
ax1.grid()

ax2 = fig2.add_subplot(2,2,2)
ax2.plot(track2_rate)
ax2.set_ylim(0, 1.1)
ax2.set_ylabel("Association Accuracy")
ax2.set_title("Track 2")
ax2.grid()

ax3 = fig2.add_subplot(2,2,3)
ax3.plot(traj_1-tracks_true[0:-1, 0:3], label=["x","y","z"])
ax3.set_xlabel("frames")
ax3.set_ylabel("position error[m]")
ax3.grid()
ax3.legend()

ax4 = fig2.add_subplot(2,2,4)
ax4.plot(traj_2-tracks_true[0:-1, 3:6], label=["x","y","z"])
ax4.set_xlabel("frames")
ax4.set_ylabel("position error[m]")
ax4.grid()
ax4.legend()

(200, 4, 2)


In [7]:
fig = plt.figure("trajectory", figsize=(10,10))

ax = plt.axes(projection="3d")

ax.plot3D(traj_1[:,0], traj_1[:,1], traj_1[:,2], marker='.', color='b')
ax.plot3D(tracks_true[:, 0], tracks_true[:, 1], tracks_true[:, 2], marker='+', color='y')

ax.plot3D(traj_2[:,0], traj_2[:,1], traj_2[:,2], marker='.', color='r')
ax.plot3D(tracks_true[:, 3], tracks_true[:, 4], tracks_true[:, 5], marker='+', color='y')

# traj_3 = np.array(track3.trajectory)
# ax.plot3D(traj_3[:,0], traj_3[:,1], traj_3[:,2], marker='.', color='r')
# # ax.plot3D(tracks_true[:, 3], tracks_true[:, 4], tracks_true[:, 5], marker='+', color='y')

ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.set_xlabel('x, m')
ax.set_ylabel('y, m')
ax.set_zlabel('z, m')




Text(0.5, 0, 'z, m')

In [14]:
fig = plt.figure("trajectory-2d", figsize=(10,10))

ax = plt.axes()
traj_1 = np.array(track1.trajectory)
ax.plot(traj_1[:,0], traj_1[:,1], marker='.', color='b', label="track 1")
ax.plot(tracks_true[:, 0], tracks_true[:, 1], marker='+', color='y', label="track 1 ground truth")
traj_2 = np.array(track2.trajectory)
ax.plot(traj_2[:,0], traj_2[:,1], marker='.', color='r', label="track 2")
ax.plot(tracks_true[:, 3], tracks_true[:, 4], marker='+', color='c', label="track 2 ground truth")
ax.set_xlim(-25,25)
ax.set_ylim(-25,25)
ax.set_xlabel('x, m')
ax.set_ylabel('y, m')
ax.grid()
ax.legend()
ax.set_title("Top-Down View of Trajectories of Two Targets")


Text(0.5, 1.0, 'Top-Down View of Trajectories of Two Targets')

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Weight visualization and Clustering", figsize=(20, 10))

ax1 = fig.add_subplot(1, 2, 1, projection="3d")
ax2 = fig.add_subplot(1, 2, 2, projection="3d")
ax1.view_init(30,0)

def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x = particles[i, :, 0]
    y = particles[i, :, 1]
    z = particles[i, :, 2]
    w = weights[i, :]
    
    ax1.cla()
    ax1.set_xlim3d(-25,25)
    ax1.set_ylim3d(-25,25)
    ax1.set_zlim3d(0,25)
    ax1.scatter(x, y, z, cmap=cm.turbo, c=w, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.set_zlabel('z, m')
    
    ax2.cla()
    ax2.set_xlim3d(-25,25)
    ax2.set_ylim3d(-25,25)
    ax2.set_zlim3d(0,25)
    ax2.set_xlabel('x, m')
    ax2.set_ylabel('y, m')
    ax2.set_zlabel('z, m')

    colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(clusters_i))]
    for cluster, col in zip(clusters_i, colors):
        ax2.scatter(
            cluster[:, 0],
            cluster[:, 1],
            cluster[:, 2],
            "o",
            color=col,
        )
        
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

In [ ]:
print(state.shape)
plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,0], state[:,1], state[:,2], marker='+', label='truth1', markersize=10)
# ax.plot3D(state[:,3], state[:,4], state[:,5], '+-', label='truth2')
ax.scatter(state_est[:,0], state_est[:,1], state_est[:,2], marker='x', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.view_init(60,-90)

plt.legend()

plt.figure(figsize=(10,8))
plt.subplot(3,1,1)
plt.plot(err1)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

plt.subplot(3,1,2)
plt.plot(n_eff)
plt.title('n_eff')
plt.xlabel('# of frames')
plt.ylabel('n_eff')

plt.subplot(3,1,3)
plt.plot(state_est[:,-1])
plt.title('label')
plt.xlabel('# of frames')
plt.ylabel('label')
plt.show()